In [ ]:
using Plots, PlotRecipes

In [ ]:
## this is a branch on my fork: tbreloff/Julia.jl
# Pkg.checkout("Julia", "tb_github")

include("/home/tom/Julia.jl/src/scrape.jl")

In [ ]:
using GitHub
myauth = authenticate("*************************************")

In [ ]:
using DataFrames
df = readtable("/home/tom/Julia.jl/db.csv", header=false, names=[:group,:subgroup,:name,:url,:desc])
head(df)

In [ ]:
function get_url_from_df(df, i)
    url = df[i,:url]
    url = join(split(url,"/")[end-1:end], "/")
end

In [ ]:
# returns (users,contributions)
function get_contributions(url)
    c = contributors(url, auth=myauth)[1]
    Plots.unzip([(get(d["contributor"].login), d["contributions"]) for d in c])
end

function get_stargazers(url)
    s = stargazers(url, auth=myauth)[1]
    [get(o.login) for o in s]
end

In [ ]:
url = "tbreloff/Plots.jl"
users, contributions = get_contributions(url)

In [ ]:
s = stargazers(url, auth=myauth)

In [ ]:
get_stargazers(url)

In [ ]:
function build_contributors_dataframe(df)
    contribs = DataFrame()
    for i = 1:4 #size(df,1)
        url = get_url_from_df(df,i)
        try
            info("trying $url")
            us,cs = get_contributions(url)
            n = length(us)
            contribs = vcat(contribs, DataFrame(
                url = fill(url,n),
                users=us,
                contribs=cs,
                group = fill(df[i,:group],n),
                subgroup = fill(df[i,:subgroup],n)
            ))
            info("success... n=$n")
        end
    end
    contribs
end

In [ ]:
contribs = build_contributors_dataframe(df)

In [ ]:
writetable("/home/tom/Julia.jl/contributors.csv", contribs)

In [ ]:
function build_stargazers_dataframe(df)
    sg = DataFrame()
    for i = 1:4 #size(df,1)
        url = get_url_from_df(df,i)
        try
            info("trying $url")
            gazers = get_stargazers(url)
            n = length(gazers)
            repo = df[i,:name][1:end-3]
            sg = vcat(sg, DataFrame(
                repo = fill(repo,n),
                url = fill(url,n),
                gazer = gazers,
                group = fill(df[i,:group],n),
                subgroup = fill(df[i,:subgroup],n)
            ))
            info("success... n=$n")
        catch err
            @show err
        end
    end
    sg
end

In [ ]:
sg = build_stargazers_dataframe(df)

In [ ]:
writetable("/home/tom/Julia.jl/stargazers.csv", sg)

In [ ]:
using Plots
zv = Plots.get_zvalues(10)
# zv = vcat(zv[1], zv[3:end])
zv = 360*zv+15
RGB[HSL(h,1,0.65) for h=zv]

In [ ]:
# this is the default Plots palette
map(RGB, Plots.get_color_palette(:auto, colorant"gray", 10))